<a href="https://colab.research.google.com/github/farahbakhsh3/DeepColorizing/blob/master/Colorizing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# %tensorflow_version 1.x
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
print('Tensorflow Version: {:s}\n********************'.format(tf.__version__))

# !nvidia-smi

In [0]:
from tensorflow.keras import layers 
from tensorflow.keras import models
from tensorflow.keras import callbacks
from tensorflow.keras import optimizers
from tensorflow.keras import datasets
from tensorflow.keras import utils
from tensorflow.keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import os

from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.transform import resize
from skimage.io import imread

In [0]:

def build2(input_shape, channels_out, drop_rate=0.3):
    
    inputs= layers.Input(shape=input_shape, name='input')
    x= inputs

    x= layers.Conv2D(8, (3, 3), strides=1, padding='same', activation='relu')(x)
    x= layers.Conv2D(16, (3, 3), strides=1, padding='same', activation='relu')(x)
    x= layers.BatchNormalization()(x)
    # x= layers.Dropout(rate=drop_rate)(x)
    x= layers.Conv2D(32, (3, 3), strides=2, padding='same', activation='relu')(x)
    x= layers.Conv2D(64, (3, 3), strides=1, padding='same', activation='relu')(x)
    x= layers.BatchNormalization()(x)
    # x= layers.Dropout(rate=drop_rate)(x)

    x= layers.Conv2DTranspose(32, (3, 3), strides=2, padding='same', activation='relu')(x)
    # x= layers.UpSampling2D()(x)
    x= layers.Conv2DTranspose(16, (3, 3), strides=1, padding='same', activation='relu')(x)
    x= layers.BatchNormalization()(x)
    # x= layers.Dropout(rate=drop_rate)(x)

    outputs= layers.Conv2DTranspose(channels_out, (3, 3), strides=1, padding='same', 
                           activation='tanh', name='decoded')(x)

    autoencoder= models.Model(inputs, outputs, name='autoencoder')
    autoencoder.summary()
    utils.plot_model(autoencoder, to_file='ColorizedModel.png',
            show_shapes=True,
            show_layer_names=False)

    return autoencoder

#===============================================

def showOrigGrayColo(img_gray, img_ab, model, ep=0, nShow=10):

    num= len(img_gray)
    s1= img_gray.shape[1]
    s2= img_gray.shape[2]

    cur= np.zeros((num, img_gray.shape[1], img_gray.shape[2], 3))
    cur[:,:,:,0]= img_gray[:,:,:,0] * 100
    cur[:,:,:,1:]= img_ab[:,:,:,0:] * 128
    orig= [lab2rgb(i) for i in cur]
    orig= np.array(orig, dtype=float)

    gray= img_gray * 100

    pred= model.predict(img_gray) 
    cur= np.zeros((num, img_gray.shape[1], img_gray.shape[2], 3))
    cur[:,:,:,0]= img_gray[:,:,:,0] * 100
    cur[:,:,:,1:]= pred * 128
    colo= [lab2rgb(i) for i in cur]
    colo= np.array(colo, dtype=float)

    x= 0
    if ep == 0:
        x= 3
        plt.figure(figsize=(num*1.5, 3))
    elif ep % nShow == 0:
        x= 1
        plt.figure(figsize=(num*1.5, 1))
    
    if x != 0:
        plt.suptitle('Epoch: {:04d}'.format(ep))

    for i in range(num):
        if x == 3:
            # display original
            ax= plt.subplot(x, num, i+1)
            plt.imshow(orig[i].reshape(s1, s2, 3))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # display gray
            ax= plt.subplot(x, num, i+1 + num)
            plt.imshow(gray[i].reshape(s1, s2), cmap='gray')
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

            # display colorized
            ax= plt.subplot(x, num, i+1 + num + num)
            plt.imshow(colo[i].reshape(s1, s2, 3))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

        elif x == 1:
            # display colorized
            ax= plt.subplot(x, num, i+1)
            plt.imshow(colo[i].reshape(s1, s2, 3))
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

    if x != 0:
        plt.show()
        # plt.savefig('./result/{:04d}.jpg'.format(ep))
    
#===============================================

def plotMinMax(Xsub_rgb,labels=['R','G','B']):
    print('__________________________________________')
    for i, lab in enumerate(labels):
        mi = np.min(Xsub_rgb[:,:,i])
        ma = np.max(Xsub_rgb[:,:,i])
        av = np.average(Xsub_rgb[:,:,i])
        print('{} : MIN={:8.4f}, MAX={:8.4f}, AVG={:8.4f}'.format(lab,mi,ma,av))

#===============================================

def makeDataset(cls=[], size=0):
    '''
    []: all class 
    0: airplane, 1: automobile, 2: bird, 
    3: cat, 4: deer, 5: dog, 6: frog, 
    7: horse, 8: ship, 9: truck
    '''    
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

    if cls:
        d_train=[]
        for idx, y in enumerate(y_train):
            if y in cls:
                d_train.append(x_train[idx])        
        x_train = np.array(d_train)
        d_test=[]
        for idx, y in enumerate(y_test):
            if y in cls: 
                d_test.append(x_test[idx])        
        x_test = np.array(d_test)

    x_train= x_train.astype('float32') / 255
    x_test= x_test.astype('float32') / 255

    if size > 0:
        x_train= x_train[:size]
        x_test= x_test[:size]

    x_train_Lab= rgb2lab(x_train)
    x_test_Lab= rgb2lab(x_test)

    x_train_gray= x_train_Lab[:,:,:,0]
    x_train_gray= x_train_gray.reshape(x_train_gray.shape+(1,))
    x_test_gray= x_test_Lab[:,:,:,0]
    x_test_gray= x_test_gray.reshape(x_test_gray.shape+(1,))
    x_train_ab= x_train_Lab[:,:,:,1:]
    x_test_ab= x_test_Lab[:,:,:,1:]
    
    x_train_gray/= 100.
    x_test_gray/= 100.
    x_train_ab/= 128.
    x_test_ab/= 128.

    return (x_train_gray, x_train_ab),(x_test_gray, x_test_ab)

In [0]:
np.random.seed(1234)
tf.random.set_seed(1234)

(x_train_gray, x_train_ab),(x_test_gray, x_test_ab) = makeDataset(cls=[7,], size=0)

img_rows= x_train_gray.shape[1]
img_cols= x_train_gray.shape[2]
channels_out= 2
channels_in= 1

input_shape= (img_rows, img_cols, channels_in)
batch_size= 32
drop_rate= 0.3
epochs= 50
init_lr= 1e-3
sample= 10
r= np.random.randint(len(x_test_gray) - sample)

# tpu= tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# strategy= tf.distribute.experimental.TPUStrategy(tpu)

# with strategy.scope():

autoencoder= build2(input_shape, channels_out, drop_rate)

# opt= optimizers.Adam(lr=init_lr, decay=init_lr / epochs)
autoencoder.compile(optimizer='adam', loss='mse')

lr_reducer= callbacks.ReduceLROnPlateau(factor=.75, cooldown=0, 
                                        patience=15, verbose=0, min_lr=1e-5)
checkpoint= callbacks.ModelCheckpoint(filepath='model_ckpt.h5', monitor='val_loss', 
                                      verbose=0, save_best_only=True)
test_callback= callbacks.LambdaCallback(on_epoch_end=lambda batch,logs: 
                                        showOrigGrayColo(x_test_gray[r:r+sample], 
                                                         x_test_ab[r:r+sample], 
                                                         autoencoder,
                                                         batch, nShow=1))
callbacks_list= [lr_reducer, checkpoint, test_callback]
# callbacks_list= [test_callback]


In [0]:
history= autoencoder.fit(x_train_gray, x_train_ab,
                validation_data=(x_test_gray, x_test_ab),
                epochs=epochs,
                batch_size=batch_size,
                verbose=1,
                callbacks=callbacks_list)

showOrigGrayColo(x_test_gray[r:r+sample], x_test_ab[r:r+sample], autoencoder)

autoencoder.save('model.h5')

In [0]:
plt.plot(history.history['loss'][:], 'r')
plt.plot(history.history['val_loss'][:], 'b')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [0]:
im_O= imread('pp.jpg')
im_O= resize(im_O,(32,32))
im_L= rgb2lab(im_O)

im_G= im_L[:,:,0]
im_G= im_G.reshape(im_G.shape+(1,))
im_AB= im_L[:,:,1:]

im_G/= 100.
im_AB/= 128.

pred= autoencoder.predict(im_G.reshape((1,) + im_G.shape)) 
cur= np.zeros((im_G.shape[0], im_G.shape[1], 3))
cur[:,:,0]= im_G[:,:,0] * 100
cur[:,:,1:]= pred * 128
colo= lab2rgb(cur)
colo= np.array(colo, dtype=float)

colo= resize(colo,(128,128))

im_O= resize(im_O,(128,128))
im_G= resize(im_G,(128,128))
im_G= im_G.reshape(128,128)

plt.figure(figsize=(10,10))
ax= plt.subplot(1, 3, 1)
plt.imshow(im_O)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax= plt.subplot(1, 3, 2)
plt.imshow(im_G, cmap='gray')
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

ax= plt.subplot(1, 3, 3)
plt.imshow(colo)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
plt.show()


In [0]:
new_sample= 50
new_r= np.random.randint(len(x_test_gray) - new_sample)
showOrigGrayColo(x_test_gray[new_r:new_r+new_sample], 
                 x_test_ab[new_r:new_r+new_sample], 
                 autoencoder)
